# Task fMRI QA Notebook
---
Notebook for visualizing quality assurance outputs of the [openfMRI subject-level task fMRI script](https://github.com/gablab/openfmri/blob/openfmri/subject_level/fmri_ants_openfmri.py):
- coregistration (functional to structural: bbregister)
- normalization (structural to MNI template: ANTS)
- ART outliers
- analysis masks (used during model fitting: FLAMEO)
- Z-stat maps
- temporal signal-to-noise ratio
- stimulus-correlated motion

The cells will display either brain slices or plots that allow for quick detection of potential problems. You must run the first 3 cells in order to use the rest of the notebook. The notebook also assumes that you used a script like dicomconvert2.py or heudiconv to convert your dicoms (the output directories of these scripts are typically named 'BOLD', 'anatomy', etc.)

See below for required packages.

(*updated Sep 3, 2016*)

In [ ]:
%matplotlib inline

import os
import glob
import fnmatch
import nilearn.plotting as nlp
import matplotlib.pyplot as plt
import nibabel as nb
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr

import warnings

### Project-specific variables to change ###

In [ ]:
openfmri_dir = '/mindhive/xnat/data/READ/openfmri/'
l1_dir = '/gablab/p/READ/openfmri_task/l1output/'
subj_prefix = 'READ_*'
fs_dir = '/mindhive/xnat/surfaces/lex/READ'   # set to None if no freesurfer dir
model_subset = ['model01']   # set to None to see all models --> if you set to None, the
                              # notebook will only look for model directories with 2 digits. 
                              # If your l1 model directories have 3 digits, list them here
skip_tasks = ['task002']   # set to empty list to see all tasks
subj_subset = ['READ_2000', 'READ_2009', 'READ_2010', 'READ_2047', 'READ_2089']
              # set to None to see all subjects
outlier_threshold = 20    # percent

### Setting up models, tasks, subject lists, runs, # of vols, contrasts, etc. ###

In [ ]:
if model_subset == None:
    models = [task.split('/')[-1] for task in sorted(glob.glob(os.path.join(l1_dir, 'model??')))]
    # grabs models in the l1_dir with just 2 digits
  
else:
    models = model_subset

def get_tasks(model):
    tasks = [task.split('/')[-1] for task in sorted(glob.glob(os.path.join(l1_dir, model, 'task*')))]
    return [task for task in tasks if task not in skip_tasks]

def get_subjlist(model, task):
    if subj_subset == None:
        subjlist = [fl for fl in sorted(os.listdir(os.path.join(l1_dir, model, task))) 
                    if fnmatch.fnmatch(fl, subj_prefix)]
    else:    
        subjlist = subj_subset
    return sorted(subjlist)

def get_subjtask(model):
    df = pd.DataFrame()
    for task in get_tasks(model):
        subjects = get_subjlist(model, task)
        df[task] = pd.Series([1 for subj in subjects], index=subjects)
    return df

def get_runs(task, subj):
    task_runs = sorted(os.listdir(os.path.join(openfmri_dir, subj, 'BOLD')))
    num_runs = 0
    for run in task_runs:
        if run.split('_')[0] == task:
            num_runs += 1
    return num_runs

def get_num_vols(task, subj, run):
    bold_file = os.path.join(openfmri_dir,subj,'BOLD','%s_run%03d' % (task,run+1),'bold.nii.gz')
    bold_img = nb.load(bold_file)
    return bold_img.shape[3]

def get_contrasts(model, task):
    contrasts_dict = {}
    model_num = int(model.split('model')[1])   # quick fix for discrepancy between 'model01' and 'model001'
    model = 'model%03d' % model_num
    task_contrasts = os.path.join(openfmri_dir, 'models', model, 'task_contrasts.txt')
    with open(task_contrasts, 'r') as f:
        contrasts = [line.split()[1] for line in f if line.split()[0]==task]
    condition_key = os.path.join(openfmri_dir, 'models', model, 'condition_key.txt')
    with open(condition_key, 'r') as f:
        conditions = [line.split()[2].strip() for line in f if line.split()[0]==task]
    with open(condition_key, 'r') as f:
        cond_gt_rest = [line.split()[2].strip() + '_gt_rest' for line in f \
                        if line.split()[0]==task]
    contrasts.extend(cond_gt_rest)
    return conditions, contrasts

# Registration #

In [ ]:
def plot_mean2anat_overlay(subj, subj_path, fs_dir, title, fig=None, ax=None):   
    if fs_dir == None:
        bg = os.path.join(openfmri_dir, subj, 'anatomy', 'T1_001.nii.gz')
    else:
        bg = os.path.join(fs_dir, subj, 'mri', 'T1.mgz')
    mask = os.path.join(os.path.join(subj_path,'qa','mask','mean2anat',
                                     'median_brain_mask.nii.gz'))
    close = False
    if ax is None:
        fig = plt.figure(figsize=(5, 2))
        ax = fig.gca()
        close = True
    display = nlp.plot_roi(roi_img=mask, bg_img=bg, black_bg=False, alpha=0.3, 
                           draw_cross=False, annotate=False,
                           figure=fig, axes=ax, title=title)
    if close:
        plt.show()
        display.close()

def plot_anat(subj_path, title, fig=None, ax=None):
    anat2target = os.path.join(subj_path, 'qa', 'anat2target', 'output_warped_image.nii.gz')
    close = False
    if ax is None:
        fig = plt.figure(figsize=(5, 2))
        ax = fig.gca()
        close = True
    display = nlp.plot_anat(anat_img=anat2target, cut_coords=(0, -13, 20), annotate=False,
                            draw_cross=False, title=title, figure=fig, axes=ax)
    if close:
        plt.show()
        display.close()

### Check registration of mean (median) functional to structural ###

Displays a mask (blue) of the coregistered median functional image overlaid on the subject's T1. Check for cases where the mask is not lined up with the brain. **Note:** this tool can only be used if you ran the version of the subject-level script that outputs a qa/mask/mean2anat directory.

In [ ]:
for model in models:
    print '********** %s **********' % model
    subj_df = get_subjtask(model)
    for subj in subj_df.index:
        fig, ax = plt.subplots(1, subj_df.shape[1], figsize=(4 * subj_df.shape[1], 2))
        if not isinstance(ax, np.ndarray):
            ax = [ax]
        for idx, task in enumerate(subj_df.ix[subj].index):
            if subj_df.ix[subj, task]:
                plot_mean2anat_overlay(subj, os.path.join(l1_dir, model, task, subj), 
                                       fs_dir, title=subj + '-' + task, fig=fig, ax=ax[idx])
        plt.show()
        plt.close(fig)

### Check registration of structural to MNI template ###

Displays cross-sections of the output of ANTS registration (the structural image warped to the MNI template). Do the brains look like MNI brains?

In [ ]:
for model in models:
    print '********** %s **********' % model
    subj_df = get_subjtask(model)
    for subj in subj_df.index:
        fig, ax = plt.subplots(1, subj_df.shape[1], figsize=(4 * subj_df.shape[1], 2))
        if not isinstance(ax, np.ndarray):
            ax = [ax]
        for idx, task in enumerate(subj_df.ix[subj].index):
            if subj_df.ix[subj, task]:
                plot_anat(os.path.join(l1_dir, model, task, subj), title=subj + '-' + task, fig=fig, ax=ax[idx])
        plt.show()
        plt.close(fig)

# Outliers #

In [ ]:
def count_outliers(subj_path, run):
    outlier_file = os.path.join(subj_path, 'qa', 'art', 
                                'run%02d_art.bold_dtype_mcf_outliers.txt' % (run + 1))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        outliers = np.genfromtxt(outlier_file)
    return np.prod(outliers.shape)

### Check percentage of outliers per run ###

Displays a heatmap of the percentage of outliers detected in each run. Darker = more outliers.

In [ ]:
outlier_dfs = {}
for model in models:
    df = pd.DataFrame()
    print '********** %s **********' % model
    tasks = get_tasks(model)
    for task in tasks:
        subjects = get_subjlist(model, task)
        max_run = 0
        subj_runs = {}
        for subj in subjects:
            num_runs = get_runs(task, subj)
            subj_runs[subj] = num_runs
            if num_runs > max_run:
                max_run = num_runs
        task_info = np.zeros((len(subjects), max_run))
        columns = ['%s-Run%02d' % (task, run) for run in range(1, max_run + 1)]
        for idx, subj in enumerate(subjects):
            for run in range(subj_runs[subj]):
                num_outliers = count_outliers(os.path.join(l1_dir, model, task, subj), run) 
                num_vols = get_num_vols(task,subj,run)
                task_info[idx, run] = float(num_outliers)/num_vols
        df_task = pd.DataFrame(task_info, index=subjects, columns=columns)
        df = pd.concat((df, df_task), axis=1)
    sns.set(context="poster", font="monospace")
    f, ax = plt.subplots(figsize=(20, 0.75*len(df.index)))
    sns.heatmap(df, linewidths=0, annot=True, vmax=outlier_threshold/100., vmin=0), #vmax=1, vmin=-1, 
    f.tight_layout()
    outlier_dfs[model] = df

### If you ran the previous cell and would like the outlier percentages outputted in csv form: ###

Indicate an output directory for the csv file below. 

In [ ]:
csv_outdir = '/path/to/csv/outdir/'  # USER OPTION

df.to_csv(os.path.join(csv_outdir, 'outlier_summary.csv'))

### Or, if you would like to see Python lists of the subjects that exceed your specified threshold: ###

In [ ]:
outlier_names = {}
for model in models:
    tasks = get_tasks(model)
    cols = outlier_dfs[model].columns.values.tolist()
    for task in tasks:
        task_cols = [col for col in cols if task in col]
        outlier_df = outlier_dfs[model][task_cols]
        outlier_names[model+'-'+task] = outlier_df.index[np.nonzero((outlier_df > outlier_threshold/100.).sum(axis=1))[0]].tolist()

outlier_names

# Masks #

In [ ]:
def plot_mask(subj_path,run):
    mask = os.path.join(subj_path,'qa','mask','run%02d_mask.nii.gz' % (run+1))
    bg = os.path.join(subj_path, 'mean', 'median.nii.gz')

    display = nlp.plot_roi(roi_img=mask, bg_img=bg, black_bg=False, alpha=0.3, display_mode='y', cut_coords=15)
    plt.show()
    display.close()

### Check analysis masks (used at FLAMEO step) for areas of poor coverage ###

Displays coronal slices of the mask (blue) overlaid on the median functional image, per run. Check for masks with poor coverage (any holes? missing sections?).

In [ ]:
for model in models:
    print '********** %s **********' % model
    tasks = get_tasks(model)
    for task in tasks:
        print '********** %s **********' % task
        for subj in get_subjlist(model, task):
            print subj
            num_runs = get_runs(task, subj)
            for run in range(num_runs):
                print 'run%02d' % (run + 1)
                plot_mask(os.path.join(l1_dir,model,task,subj),run)

# Z-Stat Maps #

In [ ]:
def plot_stat_map(subj_path, contrast_num, display_mode, threshold, title):
    zstat = os.path.join(subj_path,'zstats','mni','zstat%02d.nii.gz' % (contrast_num + 1))
    fig = plt.figure(figsize=(15, 2))
    if display_mode == 'x':
        cut_coords=np.linspace(-50, 60, 12)
    elif display_mode == 'y':
        cut_coords=np.linspace(-90, 50, 12)
    elif display_mode == 'z':
        cut_coords=np.linspace(-40, 70, 12)
    display = nlp.plot_stat_map(stat_map_img=zstat, display_mode=display_mode, threshold=threshold, 
                                figure=fig, black_bg=True, cut_coords=cut_coords, 
                                title=title, annotate=False)
    plt.show()
    display.close()

### Look through Z-stat maps in MNI space ###

Displays slices of each contrast's Z-stat map. You can change the Z threshold or the display mode below (default: Z=2.3, display_mode='z'):

'x' = sagittal;
'y' = coronal;
'z' = axial

In [ ]:
display_mode = 'z'   # USER OPTION
threshold = 2.3      # USER OPTION

for model in models:
    print '********** %s **********' % model 
    for task in get_tasks(model):
        print '********** %s **********' % task
        conditions, contrasts = get_contrasts(model,task)
        for contrast_num in range(len(contrasts)):
            print contrasts[contrast_num]
            for subj in get_subjlist(model, task):
                plot_stat_map(os.path.join(l1_dir, model, task, subj), contrast_num, 
                              display_mode, threshold, title=subj)

# Temporal Signal-to-Noise Ratio (tSNR) #

Note: this tool can only be used if you ran the subject-level script using the FreeSurfer registration workflow.

In [ ]:
def read_stats(fname):
    statsname = fname.split('_aparc')[0] + '_summary.stats'
    roi = np.genfromtxt(statsname, dtype=object)[:, 4]
    data = np.genfromtxt(fname)
    return dict(zip(roi, data))

import re

def get_data(fl, subj_regex):
    data = None
    for i, name in enumerate(fl):
        subjid = re.search(subj_regex, name).group()
        df = pd.DataFrame(read_stats(name), index=[subjid])
        if data is None:
            data = df
        else:
            data = pd.concat((data, df))
    return data.dropna(axis=1)

### Check for subjects with a tSNR profile that differs from that of other subjects ###

Displays a heatmap of the "% tSNR dissimilarity" for each subject. The subject-level script outputs a tSNR summary file with information on the mean tSNR within each of the FreeSurfer ROIs. This tool looks at the correlations between the subjects' overall tSNR profiles. Here, "% tSNR dissimilarity" refers to the percentage of subjects for which a subject had a correlation of r < 0.8 (correlations were performed within tasks/runs, and for all subjects in your l1_dir). Darker = tSNR profile is less similar. 

**You need to set the subj_regex variable to use the following cell.** subj_regex is your subject ID prefix, followed by one dot for each of the remaining characters. **Note**: this only works if all your subject IDs are the same length. 

In [ ]:
subj_regex = 'READ_....'  # USER OPTION (e.g. subject IDs are READ_1002, READ_1004, etc.)

for model in models:
    print '********** %s **********' % model 
    df = pd.DataFrame()
    for task in get_tasks(model):
        subjects = get_subjlist(model, task)
        max_run = 0
        subj_runs = {}
        for subj in subjects:
            num_runs = get_runs(task, subj)
            subj_runs[subj] = num_runs
            if num_runs > max_run:
                max_run = num_runs
        for run in range(max_run):
            fl = sorted(glob.glob(os.path.join(l1_dir, model, task, subj_prefix,
                                                'qa','tsnr','run%02d_aparc+aseg_warped_avgwf.txt' % (run+1))))
            data = get_data(fl, subj_regex)
            idx = np.nonzero(data.median(axis=0) > 20)[0]
            data_trimmed = data.ix[:, idx]
            df_task = pd.Series((np.corrcoef(data_trimmed) < 0.8).sum(axis=0), index=data.index, 
                                name=task+'-run%03d' % (run+1))
            df = pd.concat((df, df_task), axis=1)
    df = df/df.shape[0]
    col_names = df.columns.values.tolist()
    sns.set(context="poster", font="monospace")
    f, ax = plt.subplots(figsize=(20, 0.25*len(df.index)))
    sns.heatmap(df, linewidths=0, annot=True)#, #vmax=1, vmin=-1, 
    ax.tick_params(labelbottom='on',labeltop='on')
    f.tight_layout()

### Displays histograms of the "% tSNR dissimilarity" measure for each task and run: ###

In [ ]:
f, axarr = plt.subplots(len(col_names), sharex=True, sharey=True, 
                        figsize=(7,3*len(col_names)), squeeze=False)
plt.xlabel('% tSNR dissimilarity', fontsize=20)

for i, col_name in enumerate(col_names):
    axarr[i,0].hist(df.values[:,i][~np.isnan(df.values[:,i].flatten())], 30)
    axarr[i,0].set_title(col_names[i], fontsize=16)
    axarr[i,0].set_ylabel('# of participants', fontsize=14)

### Shows a Python list of the subjects that have at least 40% tSNR dissimilarity in at least one task/run:

In [ ]:
tsnr_outliers = df.index[np.nonzero((df >= .4).sum(axis=1))[0]].tolist()
tsnr_outliers

# Stimulus-Correlated Motion #

### Check for subjects with high levels of stimulus-correlated motion (e.g. greater than r=0.2-0.3) ###

It's easier to run this SCM tool one task at a time; you can change the model or task below.

In [ ]:
model = 'model01'   # USER OPTION
task = 'task001'     # USER OPTION

conditions, contrasts = get_contrasts(model, task)
n_conds = len(conditions)

cols = []
for cond_idx in range(n_conds):
    motion_cols = ['cond%s_mot%s' % (cond_idx+1, mot_idx+1) for mot_idx in range(6)]
    motion_cols.append('cond%d_outliers' % (cond_idx+1))
    cols = cols + motion_cols
    
output = pd.DataFrame()

subs_idx = []
runs_idx = []

subjects = get_subjlist(model, task)
for subj in subjects:
    corr_df = pd.DataFrame()
    num_runs = get_runs(task, subj)
    for run in range(num_runs):
        subs_idx.append(subj)
        runs_idx.append(run+1)
        multiindex = []
        multiindex = [np.array(subs_idx), np.array(runs_idx)]
        mat = pd.read_csv(os.path.join(l1_dir, model, task, subj, 'qa', 'model', 'run0%s_run%s.mat' 
                                        % (run+1, run)), delimiter='\t', skiprows=5, header=None)
        mat = mat.dropna(axis=1)
        # merging outliers into one vector
        outliers = np.genfromtxt(os.path.join(l1_dir, model, task, subj, 'qa', 'art', 
                                    'run0%s_art.bold_dtype_mcf_outliers.txt' %(run+1)), dtype='int')
        outlier_regressors = mat.ix[:,n_conds+6:len(mat.columns)]
        outliers_all = pd.DataFrame(outlier_regressors.sum(axis=1))
        # getting slice of just the conditions and motion regressors
        cond_slice = mat.ix[:,0:n_conds-1]
        mot_slice = mat.ix[:,n_conds:n_conds+5]
        mot_slice = pd.concat([mot_slice, outliers_all], axis=1, ignore_index=True)
        mat_slice = pd.concat([cond_slice, mot_slice], axis=1, ignore_index=True)
        # calculating correlation between the conditions and motion regressors
        corr_list = [pearsonr(mat_slice.ix[:,i], mat_slice.ix[:,j])[0] for i in range(n_conds) 
                        for j in range(n_conds, mat_slice.shape[1])]  
        row = pd.DataFrame([corr_list], columns=cols)
        corr_df = pd.concat([corr_df, row], ignore_index=True)
    output = pd.concat([output, corr_df], axis=0, ignore_index=True)
    
output = pd.DataFrame(output.values, index=multiindex, columns=cols)

### Make the plot here: ###

Displays a plot of the correlations (absolute value) between each condition and the 6 motion parameters used in the model. Also shows the correlations between each condition and the outlier volumes (represented here as a single time series collapsed across all outlier columns in the model).

You can take a look at the model .mat files in your qa/model l1 directories.

In [ ]:
sns.set(rc={'axes.labelsize': 25}, font_scale=1.2)
f, ax = plt.subplots(figsize=(25,len(subjects)*1.5))  # change second argument to make plot larger
plot = sns.heatmap(abs(output), vmin=.2, cbar=False, annot=True, 
                    linecolor='white', linewidth=0.5, cmap='Reds')
subs = output.index.get_level_values(0)
for i, sub in enumerate(subs):
    if i and sub != subs[i - 1]:
        ax.axhline(len(subs) - i, c="black")
conds_spacing = [x*7 for x in range(1, n_conds)]
for i in conds_spacing:
    ax.axvline(i, c="gray")
ax.tick_params(labelbottom='on',labeltop='on')
ax.set(xlabel='correlation: condition and motion regressor/outliers (threshold=0.2)', 
        ylabel='subject - run #')
plt.tight_layout()

### Are there any other QA measures you're interested in seeing? ###

Remember to clear any outputs from the notebook before saving anything.